# Lab 1 - Creating the SQL Tables

In this lab, use `sqlalchemy` to create, populate, and query a table from the baseball database, as well as for the `super_hero_powers.csv` table.  

In [1]:
import pandas as pd
artwork = pd.read_csv("./data/Artworks.csv")

## Part 1 - Baseball Managers

In this part of the lab, you will walk through the process of creating a manager table from [Lahman’s Baseball Database](http://www.seanlahman.com/baseball-archive/statistics/)

## Task 1 - Download, unzip, rename 

1. Download the baseball database linked above (save to desktop)
2. Unzip the file and rename to `baseball`
3. Load the `core/Managers.csv` file into a pandas `DataFrame` using `read_csv`
4. Inspect the `column` names and `dtypes`

In [2]:
import pandas as pd
managers = pd.read_csv('~/Desktop/baseball/core/Managers.csv')
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y


In [3]:
managers.columns

Index(['playerID', 'yearID', 'teamID', 'lgID', 'inseason', 'G', 'W', 'L',
       'rank', 'plyrMgr'],
      dtype='object')

**Question:** Is there a candidate for a primary key?

In [4]:
[(col, managers[col].is_unique) for col in managers]

[('playerID', False),
 ('yearID', False),
 ('teamID', False),
 ('lgID', False),
 ('inseason', False),
 ('G', False),
 ('W', False),
 ('L', False),
 ('rank', False),
 ('plyrMgr', False)]

**Solution:** Add the `index` as an actual column

In [5]:
from dfply import mutate
managers = (managers >>
            mutate(id = managers.index))

In [6]:
managers.id.is_unique

True

In [7]:
managers.dtypes

playerID     object
yearID        int64
teamID       object
lgID         object
inseason      int64
G             int64
W             int64
L             int64
rank        float64
plyrMgr      object
id            int64
dtype: object

In [8]:
managers.shape

(3469, 11)

In [9]:
managers.head()

,playerID,yearID,teamID,lgID,inseason,G,W,L,rank,plyrMgr,id
0,wrighha01,1871,BS1,NaN,1,31,20,10,3.0,Y,0
1,woodji01,1871,CH1,NaN,1,28,19,9,2.0,Y,1
2,paborch01,1871,CL1,NaN,1,29,10,19,8.0,Y,2
3,lennobi01,1871,FW1,NaN,1,14,5,9,8.0,Y,3
4,deaneha01,1871,FW1,NaN,2,5,2,3,8.0,Y,4


#### Task 2 - Create a `sqlalchemy` types `dict`

In [10]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID':String, 
             'plyrMgr':String,
             'teamID':String, 
             'lgID':String, 
             'yearID':Integer, 
             'inseason':Integer, 
             'G':Integer, 
             'W':Integer, 
             'L':Integer,
             'rank':Integer}

#### Task 4 - Create an `engine` and `schema`

In [11]:
!rm databases/baseball.db

In [12]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)




#### Execute the `schema`

In [13]:
mang_eng.execute(schema)

2019-01-28 08:24:24,407 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:24:24,410 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:24,415 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:24:24,418 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:24,422 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE manager (
	"playerID" VARCHAR, 
	"yearID" INTEGER, 
	"teamID" VARCHAR, 
	"lgID" VARCHAR, 
	inseason INTEGER, 
	"G" INTEGER, 
	"W" INTEGER, 
	"L" INTEGER, 
	rank INTEGER, 
	"plyrMgr" VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT manager_pk PRIMARY KEY (id)
)


2019-01-28 08:24:24,428 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:24,435 INFO sqlalchemy.engine.base.Engine COMMIT


#### Task 5 - Use `to_sql` with `if_exists='append'` to insert the data

In [14]:
managers.to_sql('manager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-28 08:24:26,987 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("manager")
2019-01-28 08:24:26,989 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:24:26,995 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:24:27,060 INFO sqlalchemy.engine.base.Engine INSERT INTO manager ("playerID", "yearID", "teamID", "lgID", inseason, "G", "W", "L", rank, "plyrMgr", id) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2019-01-28 08:24:27,061 INFO sqlalchemy.engine.base.Engine (('wrighha01', 1871, 'BS1', None, 1, 31, 20, 10, 3.0, 'Y', 0), ('woodji01', 1871, 'CH1', None, 1, 28, 19, 9, 2.0, 'Y', 1), ('paborch01', 1871, 'CL1', None, 1, 29, 10, 19, 8.0, 'Y', 2), ('lennobi01', 1871, 'FW1', None, 1, 14, 5, 9, 8.0, 'Y', 3), ('deaneha01', 1871, 'FW1', None, 2, 5, 2, 3, 8.0, 'Y', 4), ('fergubo01', 1871, 'NY2', None, 1, 33, 16, 17, 5.0, 'Y', 5), ('mcbridi01', 1871, 'PH1', None, 1, 28, 21, 7, 1.0, 'Y', 6), ('hastisc01', 1871, 'RC1', None, 1, 25, 4, 21, 9.0, 'Y', 7)  ... displaying 10 of

#### Task 6 - Query the table to make sure it all worked

In [15]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [16]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.manager

In [17]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 08:24:29,633 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:24:29,635 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM manager
2019-01-28 08:24:29,636 INFO sqlalchemy.engine.base.Engine ()


[{'G': 31,
  'L': 10,
  'W': 20,
  'id': 0,
  'inseason': 1,
  'lgID': None,
  'playerID': 'wrighha01',
  'plyrMgr': 'Y',
  'rank': 3,
  'teamID': 'BS1',
  'yearID': 1871},
 {'G': 28,
  'L': 9,
  'W': 19,
  'id': 1,
  'inseason': 1,
  'lgID': None,
  'playerID': 'woodji01',
  'plyrMgr': 'Y',
  'rank': 2,
  'teamID': 'CH1',
  'yearID': 1871},
 {'G': 29,
  'L': 19,
  'W': 10,
  'id': 2,
  'inseason': 1,
  'lgID': None,
  'playerID': 'paborch01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'CL1',
  'yearID': 1871},
 {'G': 14,
  'L': 9,
  'W': 5,
  'id': 3,
  'inseason': 1,
  'lgID': None,
  'playerID': 'lennobi01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871},
 {'G': 5,
  'L': 3,
  'W': 2,
  'id': 4,
  'inseason': 2,
  'lgID': None,
  'playerID': 'deaneha01',
  'plyrMgr': 'Y',
  'rank': 8,
  'teamID': 'FW1',
  'yearID': 1871}]

## Part 2 - Awards for Managers

Now add a table for the `AwardsManagers.csv` table.

In [20]:
import pandas as pd
amanagers = pd.read_csv('~/Desktop/baseball/core/AwardsManagers.csv')
amanagers.head()

,playerID,awardID,yearID,lgID,tie,notes
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN


In [22]:
amanagers.columns

Index(['playerID', 'awardID', 'yearID', 'lgID', 'tie', 'notes'], dtype='object')

In [23]:
[(col, amanagers[col].is_unique) for col in amanagers]

[('playerID', False),
 ('awardID', False),
 ('yearID', False),
 ('lgID', False),
 ('tie', False),
 ('notes', False)]

In [24]:
from dfply import mutate
amanagers = (amanagers >>
            mutate(id = amanagers.index))

In [26]:
amanagers.id.is_unique

True

In [27]:
amanagers.dtypes

playerID    object
awardID     object
yearID       int64
lgID        object
tie         object
notes       object
id           int64
dtype: object

In [28]:
amanagers.shape

(179, 7)

In [29]:
amanagers.head()

,playerID,awardID,yearID,lgID,tie,notes,id
0,larusto01,BBWAA Manager of the Year,1983,AL,NaN,NaN,0
1,lasorto01,BBWAA Manager of the Year,1983,NL,NaN,NaN,1
2,andersp01,BBWAA Manager of the Year,1984,AL,NaN,NaN,2
3,freyji99,BBWAA Manager of the Year,1984,NL,NaN,NaN,3
4,coxbo01,BBWAA Manager of the Year,1985,AL,NaN,NaN,4


In [30]:
from sqlalchemy import String, Integer
sql_types = {'id':Integer,
             'playerID': String, 
             'awardID': String, 
             'yearID': Integer, 
             'lgID': String, 
             'tie': String, 
             'notes': String}

In [31]:
!rm databases/baseball.db

In [32]:
from sqlalchemy import create_engine
mang_eng = create_engine("sqlite:///databases/baseball.db")
mang_eng.echo = True
schema = pd.io.sql.get_schema(amanagers, 'amanager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)


CREATE TABLE amanager (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT amanager_pk PRIMARY KEY (id)
)




In [33]:
mang_eng.execute(schema)

2019-01-28 08:31:09,356 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:31:09,358 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:09,360 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 08:31:09,361 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:09,364 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE amanager (
	"playerID" VARCHAR, 
	"awardID" VARCHAR, 
	"yearID" INTEGER, 
	"lgID" VARCHAR, 
	tie VARCHAR, 
	notes VARCHAR, 
	id INTEGER NOT NULL, 
	CONSTRAINT amanager_pk PRIMARY KEY (id)
)


2019-01-28 08:31:09,366 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:09,370 INFO sqlalchemy.engine.base.Engine COMMIT


In [34]:
amanagers.to_sql('amanager', 
                con=mang_eng, 
                dtype=sql_types, 
                index=False,
                if_exists='append')

2019-01-28 08:31:33,779 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("amanager")
2019-01-28 08:31:33,781 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 08:31:33,785 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:31:33,789 INFO sqlalchemy.engine.base.Engine INSERT INTO amanager ("playerID", "awardID", "yearID", "lgID", tie, notes, id) VALUES (?, ?, ?, ?, ?, ?, ?)
2019-01-28 08:31:33,790 INFO sqlalchemy.engine.base.Engine (('larusto01', 'BBWAA Manager of the Year', 1983, 'AL', None, None, 0), ('lasorto01', 'BBWAA Manager of the Year', 1983, 'NL', None, None, 1), ('andersp01', 'BBWAA Manager of the Year', 1984, 'AL', None, None, 2), ('freyji99', 'BBWAA Manager of the Year', 1984, 'NL', None, None, 3), ('coxbo01', 'BBWAA Manager of the Year', 1985, 'AL', None, None, 4), ('herzowh01', 'BBWAA Manager of the Year', 1985, 'NL', None, None, 5), ('mcnamjo99', 'BBWAA Manager of the Year', 1986, 'AL', None, None, 6), ('lanieha01', 'BBWAA Manager of the Year', 1986, 'NL

In [35]:
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

mang_eng2 = create_engine("sqlite:///databases/baseball.db") 
Session = sessionmaker(mang_eng)
session = Session()

In [36]:
Base = automap_base()
Base.prepare(mang_eng2, reflect=True)
Manager = Base.classes.amanager

In [38]:
from more_sqlalchemy import result_dicts
stmt = select('*').select_from(Manager)
session.execute(stmt).fetchmany(5) >> result_dicts

2019-01-28 08:33:18,576 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-28 08:33:18,578 INFO sqlalchemy.engine.base.Engine SELECT * 
FROM amanager
2019-01-28 08:33:18,580 INFO sqlalchemy.engine.base.Engine ()


[{'awardID': 'BBWAA Manager of the Year',
  'id': 0,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'larusto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 1,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'lasorto01',
  'tie': None,
  'yearID': 1983},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 2,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'andersp01',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 3,
  'lgID': 'NL',
  'notes': None,
  'playerID': 'freyji99',
  'tie': None,
  'yearID': 1984},
 {'awardID': 'BBWAA Manager of the Year',
  'id': 4,
  'lgID': 'AL',
  'notes': None,
  'playerID': 'coxbo01',
  'tie': None,
  'yearID': 1985}]

## Part 3 - Super Hero Powers

Now make a database and table for the super hero powers.

## Problem 1
    
**Task:** One the `super_hero_powers.csv` and verify that the contents of the columns are all Boolean.  In this problem, you need to

1. Create a `dict` that defines the `pandas` column type
2. Read the file in using a `pd.read_csv`.
3. Clean up all the column labels.
    
**Be sure to write clean code!**


In [2]:
import pandas as pd
powers = pd.read_csv("./data/super_hero_powers.csv")

In [3]:
powers.head()

,hero_names,Agility,Accelerated Healing,Lantern Power Ring,Dimensional Awareness,Cold Resistance,Durability,Stealth,Energy Absorption,Flight,...,Web Creation,Reality Warping,Odin Force,Symbiote Costume,Speed Force,Phoenix Force,Molecular Dissipation,Vision - Cryo,Omnipresent,Omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [4]:
powers.columns

Index(['hero_names', 'Agility', 'Accelerated Healing', 'Lantern Power Ring',
       'Dimensional Awareness', 'Cold Resistance', 'Durability', 'Stealth',
       'Energy Absorption', 'Flight',
       ...
       'Web Creation', 'Reality Warping', 'Odin Force', 'Symbiote Costume',
       'Speed Force', 'Phoenix Force', 'Molecular Dissipation',
       'Vision - Cryo', 'Omnipresent', 'Omniscient'],
      dtype='object', length=168)

In [5]:
[(col, powers[col].is_unique) for col in powers]

[('hero_names', True),
 ('Agility', False),
 ('Accelerated Healing', False),
 ('Lantern Power Ring', False),
 ('Dimensional Awareness', False),
 ('Cold Resistance', False),
 ('Durability', False),
 ('Stealth', False),
 ('Energy Absorption', False),
 ('Flight', False),
 ('Danger Sense', False),
 ('Underwater breathing', False),
 ('Marksmanship', False),
 ('Weapons Master', False),
 ('Power Augmentation', False),
 ('Animal Attributes', False),
 ('Longevity', False),
 ('Intelligence', False),
 ('Super Strength', False),
 ('Cryokinesis', False),
 ('Telepathy', False),
 ('Energy Armor', False),
 ('Energy Blasts', False),
 ('Duplication', False),
 ('Size Changing', False),
 ('Density Control', False),
 ('Stamina', False),
 ('Astral Travel', False),
 ('Audio Control', False),
 ('Dexterity', False),
 ('Omnitrix', False),
 ('Super Speed', False),
 ('Possession', False),
 ('Animal Oriented Powers', False),
 ('Weapon-based Powers', False),
 ('Electrokinesis', False),
 ('Darkforce Manipulation', F

In [6]:
from dfply import *
clean_names = lambda name: name.strip().lower().replace(' ', '_')
new_names = {clean_names(old_name):old_name for old_name in powers.columns}
new_names

{'accelerated_healing': 'Accelerated Healing',
 'adaptation': 'Adaptation',
 'agility': 'Agility',
 'animal_attributes': 'Animal Attributes',
 'animal_control': 'Animal Control',
 'animal_oriented_powers': 'Animal Oriented Powers',
 'animation': 'Animation',
 'anti-gravity': 'Anti-Gravity',
 'astral_projection': 'Astral Projection',
 'astral_travel': 'Astral Travel',
 'audio_control': 'Audio Control',
 'banish': 'Banish',
 'biokinesis': 'Biokinesis',
 'camouflage': 'Camouflage',
 'changing_armor': 'Changing Armor',
 'clairvoyance': 'Clairvoyance',
 'cloaking': 'Cloaking',
 'cold_resistance': 'Cold Resistance',
 'cryokinesis': 'Cryokinesis',
 'danger_sense': 'Danger Sense',
 'darkforce_manipulation': 'Darkforce Manipulation',
 'death_touch': 'Death Touch',
 'density_control': 'Density Control',
 'dexterity': 'Dexterity',
 'dimensional_awareness': 'Dimensional Awareness',
 'dimensional_travel': 'Dimensional Travel',
 'duplication': 'Duplication',
 'durability': 'Durability',
 'echolocati

In [7]:
from dfply import rename
powers_renamed = (powers >>
              rename(**new_names))
powers_renamed.columns

Index(['hero_names', 'agility', 'accelerated_healing', 'lantern_power_ring',
       'dimensional_awareness', 'cold_resistance', 'durability', 'stealth',
       'energy_absorption', 'flight',
       ...
       'web_creation', 'reality_warping', 'odin_force', 'symbiote_costume',
       'speed_force', 'phoenix_force', 'molecular_dissipation',
       'vision_-_cryo', 'omnipresent', 'omniscient'],
      dtype='object', length=168)

In [8]:
powers_renamed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667 entries, 0 to 666
Columns: 168 entries, hero_names to omniscient
dtypes: bool(167), object(1)
memory usage: 114.1+ KB


In [9]:
powers_renamed.dtypes

hero_names               object
agility                    bool
accelerated_healing        bool
lantern_power_ring         bool
dimensional_awareness      bool
cold_resistance            bool
durability                 bool
stealth                    bool
energy_absorption          bool
flight                     bool
danger_sense               bool
underwater_breathing       bool
marksmanship               bool
weapons_master             bool
power_augmentation         bool
animal_attributes          bool
longevity                  bool
intelligence               bool
super_strength             bool
cryokinesis                bool
telepathy                  bool
energy_armor               bool
energy_blasts              bool
duplication                bool
size_changing              bool
density_control            bool
stamina                    bool
astral_travel              bool
audio_control              bool
dexterity                  bool
                          ...  
intuitiv

In [10]:
powers_renamed.head()

,hero_names,agility,accelerated_healing,lantern_power_ring,dimensional_awareness,cold_resistance,durability,stealth,energy_absorption,flight,...,web_creation,reality_warping,odin_force,symbiote_costume,speed_force,phoenix_force,molecular_dissipation,vision_-_cryo,omnipresent,omniscient
0,3-D Man,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,A-Bomb,False,True,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Abe Sapien,True,True,False,False,True,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,Abin Sur,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Abomination,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [23]:
from sqlalchemy import String, Boolean
sql_types = {'hero_names': String,
            'agility':Boolean,
             'accelerated_healing':Boolean,
             'lantern_power_ring':Boolean,
             'dimensional_awareness':Boolean,
             'cold_resistance':Boolean,
             'durability':Boolean,
             'stealth':Boolean,
             'energy_absorption':Boolean,
             'flight':Boolean,
             'danger_sense':Boolean,
             'underwater_breathing':Boolean,
             'marksmanship':Boolean,
             'weapons_master':Boolean,
             'power_augmentation':Boolean,
             'animal_attributes':Boolean,
             'longevity':Boolean,
             'intelligence':Boolean,
             'super_strength':Boolean,
             'cryokinesis':Boolean,
             'telepathy':Boolean,
             'energy_armor':Boolean,
             'energy_blasts':Boolean,
             'duplication':Boolean,
             'size_changing':Boolean,
             'density_control':Boolean,
             'stamina':Boolean,
             'astral_travel':Boolean,
             'audio_control':Boolean,
             'dexterity':Boolean,
             'omnitrix':Boolean,
             'super_speed':Boolean,
             'possession':Boolean,
             'animal_oriented_powers':Boolean,
             'weapon-based_powers':Boolean,
             'electrokinesis':Boolean,
             'darkforce_manipulation':Boolean,
             'death_touch':Boolean,
             'teleportation':Boolean,
             'enhanced_senses':Boolean,
             'telekinesis':Boolean,
             'energy_beams':Boolean,
             'magic':Boolean,
             'hyperkinesis':Boolean,
             'jump':Boolean,
             'clairvoyance':Boolean,
             'dimensional_travel':Boolean,
             'power_sense':Boolean,
             'shapeshifting':Boolean,
             'peak_human_condition':Boolean,
             'immortality':Boolean,
             'camouflage':Boolean,
             'element_control':Boolean,
             'phasing':Boolean,
             'astral_projection':Boolean,
             'electrical_transport':Boolean,
             'fire_control':Boolean,
             'projection':Boolean,
             'summoning':Boolean,
             'enhanced_memory':Boolean,
             'reflexes':Boolean,
             'invulnerability':Boolean,
             'energy_constructs':Boolean,
             'force_fields':Boolean,
             'self-sustenance':Boolean,
             'anti-gravity':Boolean,
             'empathy':Boolean,
             'power_nullifier':Boolean,
             'radiation_control':Boolean,
             'psionic_powers':Boolean,
             'elasticity':Boolean,
             'substance_secretion':Boolean,
             'elemental_transmogrification':Boolean,
             'technopath/cyberpath':Boolean,
             'photographic_reflexes':Boolean,
             'seismic_power':Boolean,
             'animation':Boolean,
             'precognition':Boolean,
             'mind_control':Boolean,
             'fire_resistance':Boolean,
             'power_absorption':Boolean,
             'enhanced_hearing':Boolean,
             'nova_force':Boolean,
             'insanity':Boolean,
             'hypnokinesis':Boolean,
             'animal_control':Boolean,
             'natural_armor':Boolean,
             'intangibility':Boolean,
             'enhanced_sight':Boolean,
             'molecular_manipulation':Boolean,
             'heat_generation':Boolean,
             'adaptation':Boolean,
             'gliding':Boolean,
             'power_suit':Boolean,
             'mind_blast':Boolean,
             'probability_manipulation':Boolean,
             'gravity_control':Boolean,
             'regeneration':Boolean,
             'light_control':Boolean,
             'echolocation':Boolean,
             'levitation':Boolean,
             'toxin_and_disease_control':Boolean,
             'banish':Boolean,
             'energy_manipulation':Boolean,
             'heat_resistance':Boolean,
             'natural_weapons':Boolean,
             'time_travel':Boolean,
             'enhanced_smell':Boolean,
             'illusions':Boolean,
             'thirstokinesis':Boolean,
             'hair_manipulation':Boolean,
             'illumination':Boolean,
             'omnipotent':Boolean,
             'cloaking':Boolean,
             'changing_armor':Boolean,
             'power_cosmic':Boolean,
             'biokinesis':Boolean,
             'water_control':Boolean,
             'radiation_immunity':Boolean,
             'vision_-_telescopic':Boolean,
             'toxin_and_disease_resistance':Boolean,
             'spatial_awareness':Boolean,
             'energy_resistance':Boolean,
             'telepathy_resistance':Boolean,
             'molecular_combustion':Boolean,
             'omnilingualism':Boolean,
             'portal_creation':Boolean,
             'magnetism':Boolean,
             'mind_control_resistance':Boolean,
             'plant_control':Boolean,
             'sonar':Boolean,
             'sonic_scream':Boolean,
             'time_manipulation':Boolean,
             'enhanced_touch':Boolean,
             'magic_resistance':Boolean,
             'invisibility':Boolean,
             'sub-mariner':Boolean,
             'radiation_absorption':Boolean,
             'intuitive_aptitude':Boolean,
             'vision_-_microscopic':Boolean,
             'melting':Boolean,
             'wind_control':Boolean,
             'super_breath':Boolean,
             'wallcrawling':Boolean,
             'vision_-_night':Boolean,
             'vision_-_infrared':Boolean,
             'grim_reaping':Boolean,
             'matter_absorption':Boolean,
             'the_force':Boolean,
             'resurrection':Boolean,
             'terrakinesis':Boolean,
             'vision_-_heat':Boolean,
             'vitakinesis':Boolean,
             'radar_sense':Boolean,
             'qwardian_power_ring':Boolean,
             'weather_control':Boolean,
             'vision_-_x-ray':Boolean,
             'vision_-_thermal':Boolean,
             'web_creation':Boolean,
             'reality_warping':Boolean,
             'odin_force':Boolean,
             'symbiote_costume':Boolean,
             'speed_force':Boolean,
             'phoenix_force':Boolean,
             'molecular_dissipation':Boolean,
             'vision_-_cryo':Boolean,
             'omnipresent':Boolean,
             'omniscient':Boolean
              }

## Problem 2
    
Now define an `sqlalchemy` table for these data using `pandas` `to_sql` dataframe method.  You can use the `sqlalchemy.String` and `sqlalchemy.Boolean` columns type, which are [documented here](https://docs.sqlalchemy.org/en/latest/core/type_basics.html)

## Problem 3
    
Now you need to make a new `engine`, `inspect` your database, and make a `session` to query the database.

In [ ]:
from sqlalchemy import create_engine
power_eng = create_engine("sqlite:///databases/powers.db")
power_eng.echo = True
schema = pd.io.sql.get_schema(managers, 'manager', keys='id', con=mang_eng, dtype=sql_types)
print(schema)

## Problem 4
    
Perform `sqlalchemy` queries to answer each of the following questions.

1. How many heroes have both Super Strength and Super Speed?
2. How many heroes have names that start with the word *Black*
3. Are heroes with Agility more likely to have Stealth?
4. What fraction of all heroes that can fly also have Super Strength?
5. Consider heroes that have names that contain `"girl"`, `"boy"`, `"woman"`, or `"man"`.  Compute the following ratio

$$\frac{N(\text{boy or man})}{N(\text{girl or woman}}$$

**Hint:** You will need to use some combination of `where`, `group_by`, and `count` for each part.

## Problem 5

Tell me another cool fact about the super powers.